# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [1]:
# one-hot encode the desiredd outputs: [number, 'fizz', 'buzz', 'fizzbuzz']

def fizz_buzz_encode(i):
    if i%15 ==0:
        return 3
    elif i%5 ==0:
        return 2
    elif i%3 ==0:
        return 1
    else:
        return 0

def fizz_buzz_decode(i, prediction):
    return [str(i), 'fizz', 'buzz', 'fizzbuzz'][prediction]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


# 使用神经网络进行预测

### 定义Dataset  
由于数据为10进制数时效果较差，故将数据设为2进制

In [2]:
import numpy as np
import torch

NUM_DIGITS=10

def binary_encode(i):
    bin_i=bin(i)
    bin_i_10=(NUM_DIGITS+2-len(bin_i))*'0'+bin_i[2:]
    bin_i_10_toarray=np.array([int(i) for i in bin_i_10])
    return bin_i_10_toarray

# binary_encode(31)

#1-100为测试数据 101-end为训练数据
train_x=torch.Tensor([binary_encode(i) for i in range(101, 2**NUM_DIGITS)])
train_y=torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2**NUM_DIGITS)]) #y为整型

# print(train_x)
# print(train_x.shape)
# print(train_y)
# print(train_y.shape)

### 定义模型

In [3]:
#定义device
device=torch.device('cuda')
#定义隐含层神经元个数
NUM_HIDDEN=100
#定义learning rate
le_rate=0.05

#将参数传入gpu
if torch.cuda.is_available():
    train_x=train_x.to(device)
    train_y=train_y.to(device)

#定义模型
model=torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS,NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN,4),
)

#定义lossfunction
loss_fn=torch.nn.CrossEntropyLoss()
#定义optimizer
optimizer=torch.optim.SGD(model.parameters(),lr=le_rate)

#model移入gpu
if torch.cuda.is_available():
    model=model.to(device)

### 训练代码

In [4]:
#定义batch size
BATCH_SIZE=128

# list(range(0, train_x.shape[0], BATCH_SIZE))
#[0, 128, 256, 384, 512, 640, 768, 896]
#每个epoch做8次训练，顺序训练训练集的8组数据，每组128个数据
for epoch in range(10000):
    for start in range(0, len(train_x), BATCH_SIZE):
        end=start+BATCH_SIZE
        batch_x=train_x[start:end]
        batch_y=train_y[start:end]
        
        #calc model
        y_pred=model(batch_x)
        # print(batch_y)
        # print(batch_y.shape)
        # print(y_pred)
        # print(y_pred.shape)
        
        #calc loss
        loss=loss_fn(y_pred,batch_y)
        print('Epoch', epoch+1, loss.item())
        
        #loss backward
        loss.backward()
        
        #update model parameters
        optimizer.step()
        
        #清空grad
        optimizer.zero_grad()

Epoch 1 1.3452568054199219
Epoch 1 1.3311318159103394
Epoch 1 1.28657066822052
Epoch 1 1.251975655555725
Epoch 1 1.2158681154251099
Epoch 1 1.2100368738174438
Epoch 1 1.1988413333892822
Epoch 1 1.1285687685012817
Epoch 2 1.1871784925460815
Epoch 2 1.1958396434783936
Epoch 2 1.1699919700622559
Epoch 2 1.1805994510650635
Epoch 2 1.155923843383789
Epoch 2 1.1552789211273193
Epoch 2 1.1606690883636475
Epoch 2 1.112045168876648
Epoch 3 1.1593033075332642
Epoch 3 1.1714041233062744
Epoch 3 1.1491875648498535
Epoch 3 1.1676095724105835
Epoch 3 1.1453208923339844
Epoch 3 1.143805742263794
Epoch 3 1.1532528400421143
Epoch 3 1.1122443675994873
Epoch 4 1.1506322622299194
Epoch 4 1.1635767221450806
Epoch 4 1.142643928527832
Epoch 4 1.16336190700531
Epoch 4 1.1420811414718628
Epoch 4 1.1398602724075317
Epoch 4 1.1508584022521973
Epoch 4 1.1134979724884033
Epoch 5 1.146893858909607
Epoch 5 1.1600488424301147
Epoch 5 1.1399199962615967
Epoch 5 1.1614538431167603
Epoch 5 1.1407573223114014
Epoch 5 1.1

Epoch 42 1.135440707206726
Epoch 42 1.145345687866211
Epoch 42 1.1329885721206665
Epoch 42 1.1521350145339966
Epoch 42 1.1359918117523193
Epoch 42 1.1325944662094116
Epoch 42 1.1440831422805786
Epoch 42 1.0879958868026733
Epoch 43 1.135327935218811
Epoch 43 1.1451871395111084
Epoch 43 1.132927656173706
Epoch 43 1.1520097255706787
Epoch 43 1.1359485387802124
Epoch 43 1.13253915309906
Epoch 43 1.1440331935882568
Epoch 43 1.0874428749084473
Epoch 44 1.1352132558822632
Epoch 44 1.1450272798538208
Epoch 44 1.1328656673431396
Epoch 44 1.151892066001892
Epoch 44 1.135910987854004
Epoch 44 1.132483720779419
Epoch 44 1.1439802646636963
Epoch 44 1.0868802070617676
Epoch 45 1.1350983381271362
Epoch 45 1.1448715925216675
Epoch 45 1.1328006982803345
Epoch 45 1.151774525642395
Epoch 45 1.1358730792999268
Epoch 45 1.132430911064148
Epoch 45 1.1439288854599
Epoch 45 1.0863419771194458
Epoch 46 1.1349910497665405
Epoch 46 1.1447232961654663
Epoch 46 1.1327468156814575
Epoch 46 1.1516578197479248
Epoch 

Epoch 86 1.1304221153259277
Epoch 86 1.1427927017211914
Epoch 86 1.0687037706375122
Epoch 87 1.1313668489456177
Epoch 87 1.1400213241577148
Epoch 87 1.1310276985168457
Epoch 87 1.1474823951721191
Epoch 87 1.1344906091690063
Epoch 87 1.130370855331421
Epoch 87 1.1427727937698364
Epoch 87 1.0683505535125732
Epoch 88 1.131286382675171
Epoch 88 1.139927625656128
Epoch 88 1.130994439125061
Epoch 88 1.1473907232284546
Epoch 88 1.1344465017318726
Epoch 88 1.1303316354751587
Epoch 88 1.1427638530731201
Epoch 88 1.067970871925354
Epoch 89 1.131203532218933
Epoch 89 1.1398346424102783
Epoch 89 1.1309568881988525
Epoch 89 1.147290587425232
Epoch 89 1.1344075202941895
Epoch 89 1.1302796602249146
Epoch 89 1.1427487134933472
Epoch 89 1.0676236152648926
Epoch 90 1.1311250925064087
Epoch 90 1.1397407054901123
Epoch 90 1.130915880203247
Epoch 90 1.147193193435669
Epoch 90 1.1343742609024048
Epoch 90 1.1302361488342285
Epoch 90 1.1427383422851562
Epoch 90 1.0672491788864136
Epoch 91 1.1310399770736694
E

Epoch 130 1.143011212348938
Epoch 130 1.1325432062149048
Epoch 130 1.127941370010376
Epoch 130 1.1419700384140015
Epoch 130 1.0536108016967773
Epoch 131 1.127413034439087
Epoch 131 1.135466456413269
Epoch 131 1.1290048360824585
Epoch 131 1.1428874731063843
Epoch 131 1.132482886314392
Epoch 131 1.1278830766677856
Epoch 131 1.141945481300354
Epoch 131 1.0532724857330322
Epoch 132 1.1272969245910645
Epoch 132 1.1353482007980347
Epoch 132 1.1289281845092773
Epoch 132 1.1427710056304932
Epoch 132 1.1324286460876465
Epoch 132 1.1278053522109985
Epoch 132 1.1419241428375244
Epoch 132 1.0529711246490479
Epoch 133 1.1272274255752563
Epoch 133 1.13520348072052
Epoch 133 1.128867745399475
Epoch 133 1.1426454782485962
Epoch 133 1.1323621273040771
Epoch 133 1.1277611255645752
Epoch 133 1.1418962478637695
Epoch 133 1.0526303052902222
Epoch 134 1.12711501121521
Epoch 134 1.1351016759872437
Epoch 134 1.1288025379180908
Epoch 134 1.1425440311431885
Epoch 134 1.1323267221450806
Epoch 134 1.1276816129684

Epoch 174 1.1247332096099854
Epoch 174 1.1404902935028076
Epoch 174 1.0390082597732544
Epoch 175 1.122649908065796
Epoch 175 1.1295325756072998
Epoch 175 1.125903844833374
Epoch 175 1.1373145580291748
Epoch 175 1.128990888595581
Epoch 175 1.1246410608291626
Epoch 175 1.1404420137405396
Epoch 175 1.038612723350525
Epoch 176 1.1225826740264893
Epoch 176 1.1293808221817017
Epoch 176 1.1258050203323364
Epoch 176 1.137186884880066
Epoch 176 1.128857970237732
Epoch 176 1.1245931386947632
Epoch 176 1.140396237373352
Epoch 176 1.0382999181747437
Epoch 177 1.122403621673584
Epoch 177 1.1292603015899658
Epoch 177 1.125718593597412
Epoch 177 1.137040376663208
Epoch 177 1.1287667751312256
Epoch 177 1.1244655847549438
Epoch 177 1.1403181552886963
Epoch 177 1.0379581451416016
Epoch 178 1.1223078966140747
Epoch 178 1.1290857791900635
Epoch 178 1.1256276369094849
Epoch 178 1.136897325515747
Epoch 178 1.12864351272583
Epoch 178 1.1243852376937866
Epoch 178 1.1402807235717773
Epoch 178 1.037534952163696

...

Epoch 9813 0.008189568296074867
Epoch 9813 0.009658191353082657
Epoch 9813 0.0014594042440876365
Epoch 9814 0.005315572023391724
Epoch 9814 0.006020832806825638
Epoch 9814 0.00530879944562912
Epoch 9814 0.005572035908699036
Epoch 9814 0.009781986474990845
Epoch 9814 0.008186334744095802
Epoch 9814 0.00965014286339283
Epoch 9814 0.0014650026569142938
Epoch 9815 0.005311500281095505
Epoch 9815 0.0060219839215278625
Epoch 9815 0.0053120870143175125
Epoch 9815 0.005572134628891945
Epoch 9815 0.009777937084436417
Epoch 9815 0.008185885846614838
Epoch 9815 0.009657388553023338
Epoch 9815 0.0014614352257922292
Epoch 9816 0.005311202257871628
Epoch 9816 0.0060186125338077545
Epoch 9816 0.005311025306582451
Epoch 9816 0.005570610985159874
Epoch 9816 0.009775981307029724
Epoch 9816 0.0081839170306921
Epoch 9816 0.009647568687796593
Epoch 9816 0.001453947159461677
Epoch 9817 0.0053163617849349976
Epoch 9817 0.006016455590724945
Epoch 9817 0.005306927487254143
Epoch 9817 0.005569884553551674
Epoch

Epoch 9859 0.001446600304916501
Epoch 9860 0.005275014787912369
Epoch 9860 0.0059739090502262115
Epoch 9860 0.005277717486023903
Epoch 9860 0.005531756207346916
Epoch 9860 0.009710755199193954
Epoch 9860 0.0081245806068182
Epoch 9860 0.009586788713932037
Epoch 9860 0.0014526578597724438
Epoch 9861 0.005275808274745941
Epoch 9861 0.005974132567644119
Epoch 9861 0.005275908857584
Epoch 9861 0.005531197413802147
Epoch 9861 0.009708937257528305
Epoch 9861 0.008123718202114105
Epoch 9861 0.009586663916707039
Epoch 9861 0.0014463000698015094
Epoch 9862 0.005274839699268341
Epoch 9862 0.005972601473331451
Epoch 9862 0.005273682996630669
Epoch 9862 0.005531001836061478
Epoch 9862 0.00970638170838356
Epoch 9862 0.008122261613607407
Epoch 9862 0.009580228477716446
Epoch 9862 0.001451527583412826
Epoch 9863 0.005272421985864639
Epoch 9863 0.005972612649202347
Epoch 9863 0.005275106057524681
Epoch 9863 0.005530083552002907
Epoch 9863 0.00970497541129589
Epoch 9863 0.008122753351926804
Epoch 9863 0

Epoch 9904 0.008068043738603592
Epoch 9904 0.00952553004026413
Epoch 9904 0.001434944337233901
Epoch 9905 0.005239807069301605
Epoch 9905 0.00593075156211853
Epoch 9905 0.005240654572844505
Epoch 9905 0.005491157993674278
Epoch 9905 0.00964355282485485
Epoch 9905 0.008064484223723412
Epoch 9905 0.00951995700597763
Epoch 9905 0.0014408252900466323
Epoch 9906 0.005237143486738205
Epoch 9906 0.005929112434387207
Epoch 9906 0.005241883918642998
Epoch 9906 0.005492370575666428
Epoch 9906 0.00964200496673584
Epoch 9906 0.008067699149250984
Epoch 9906 0.009521789848804474
Epoch 9906 0.0014366573886945844
Epoch 9907 0.005237683653831482
Epoch 9907 0.005927402526140213
Epoch 9907 0.005240652710199356
Epoch 9907 0.0054902732372283936
Epoch 9907 0.009639831259846687
Epoch 9907 0.008064066991209984
Epoch 9907 0.009522275999188423
Epoch 9907 0.001442220527678728
Epoch 9908 0.005235336720943451
Epoch 9908 0.0059290193021297455
Epoch 9908 0.005239132791757584
Epoch 9908 0.005490405485033989
Epoch 990

Epoch 9949 0.0054528191685676575
Epoch 9949 0.009576845914125443
Epoch 9949 0.008010488003492355
Epoch 9949 0.00946241244673729
Epoch 9949 0.001433284138329327
Epoch 9950 0.005200929939746857
Epoch 9950 0.005889754742383957
Epoch 9950 0.005206400528550148
Epoch 9950 0.005453683435916901
Epoch 9950 0.009572897106409073
Epoch 9950 0.008007694035768509
Epoch 9950 0.009459735825657845
Epoch 9950 0.0014283391647040844
Epoch 9951 0.005200430750846863
Epoch 9951 0.005885791033506393
Epoch 9951 0.005205048248171806
Epoch 9951 0.0054522547870874405
Epoch 9951 0.009574156254529953
Epoch 9951 0.00800706073641777
Epoch 9951 0.009458787739276886
Epoch 9951 0.001427244278602302
Epoch 9952 0.005202118307352066
Epoch 9952 0.005884416401386261
Epoch 9952 0.0052028801292181015
Epoch 9952 0.005451682955026627
Epoch 9952 0.009570611640810966
Epoch 9952 0.008005935698747635
Epoch 9952 0.009453224018216133
Epoch 9952 0.0014232175890356302
Epoch 9953 0.005202990025281906
Epoch 9953 0.0058844611048698425
Epoc

Epoch 9993 0.007953472435474396
Epoch 9993 0.00940069742500782
Epoch 9993 0.0014232000103220344
Epoch 9994 0.005165599286556244
Epoch 9994 0.00584610179066658
Epoch 9994 0.005172833800315857
Epoch 9994 0.005415584892034531
Epoch 9994 0.009511396288871765
Epoch 9994 0.007952658459544182
Epoch 9994 0.009400790557265282
Epoch 9994 0.0014145285822451115
Epoch 9995 0.005170263350009918
Epoch 9995 0.005844246596097946
Epoch 9995 0.005169220268726349
Epoch 9995 0.005414729937911034
Epoch 9995 0.009508371353149414
Epoch 9995 0.007950594648718834
Epoch 9995 0.00940038077533245
Epoch 9995 0.0014209747314453125
Epoch 9996 0.005166336894035339
Epoch 9996 0.005846157670021057
Epoch 9996 0.005169292911887169
Epoch 9996 0.0054142046719789505
Epoch 9996 0.0095087680965662
Epoch 9996 0.007948581129312515
Epoch 9996 0.00939098559319973
Epoch 9996 0.0014156588586047292
Epoch 9997 0.005164559930562973
Epoch 9997 0.005842432379722595
Epoch 9997 0.005170697346329689
Epoch 9997 0.005411578342318535
Epoch 999

### 测试模型

In [27]:
test_x=torch.Tensor([binary_encode(i) for i in range(1, 101)])
test_y=torch.LongTensor([fizz_buzz_encode(i) for i in range(1, 101)])
# print(test_y)

#use gpu
if torch.cuda.is_available():
    test_x=test_x.to(device)


#测试不需要grad
with torch.no_grad():
    test_y_pred=model(test_x)

#处理预测结果
# print(test_y_pred)
max_test_y_pred=test_y_pred.max(1) #求每行的最大值 返回最大值与最大值的位置
# print(max_test_y_pred)
position_max_test_y_pred=max_test_y_pred[1]
# print(position_max_test_y_pred)

compare=zip(test_y.tolist(),position_max_test_y_pred.cpu().tolist())

result=[x==y for x,y in compare]
print(result)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
